In [1]:
import pandas as pd

In [2]:
import ast  # to safely convert string lists into real Python lists

df = pd.read_csv("all_predictions_detailed.csv", converters={"categories": ast.literal_eval})
df.head()

,Unnamed: 0,app,content,score,at,aspects,opinions,sentiments,categories
0,0,Al Habib,good,5,2025-07-11 10:24:50,['NULL'],['good'],['positive'],[LAPTOP#GENERAL]
1,1,Al Habib,Its useless i am trying to register my account...,1,2025-07-11 05:53:23,"['email error', 'helpline']","['useless', 'NULL']","['negative', 'negative']","[SUPPORT#GENERAL, SUPPORT#GENERAL]"
2,2,Al Habib,app is good but I have an issue now I'm living...,5,2025-07-10 23:28:19,['app'],['good'],['positive'],[SOFTWARE#GENERAL]
3,3,Al Habib,bank is good coie sarvis,5,2025-07-10 21:49:23,['bank'],['good'],['positive'],[SUPPORT#GENERAL]
4,4,Al Habib,There is so much wrong about this net banking ...,1,2025-07-10 18:01:09,"['net banking app', 'biometric problem', 'BAHL...","['wrong', 'NULL', 'worst decision']","['negative', 'negative', 'negative']","[SOFTWARE#GENERAL, SOFTWARE#GENERAL, SOFTWARE#..."


In [3]:
# Flatten all categories
all_categories = [cat for sublist in df["categories"] for cat in sublist]

# Convert to a unique sorted list
unique_categories = sorted(set(all_categories))

# Display
print("Total unique categories:", len(unique_categories))
for cat in unique_categories:
    print(cat)

Total unique categories: 54

AMBIENCE#GENERAL
CHARTER#GENERAL
CNIC#GENERAL
COMPANY#GENERAL
COMplaint#OPERATION_PERFORMANCE
COSTS#GENERAL
CURRENCY#GENERAL
CURRENCY#OPERATION_PERFORMANCE
DISPLAY#GENERAL
DISPLAY#OPERATION_PERFORMANCE
DISPLAY#USABILITY
DRINKS#OPERATION_PERFORMANCE
FOOD#QUALITY
Font#OPERATION_PERFORMANCE
HARDWARE#GENERAL
HARDWARE#OPERATION_PERFORMANCE
KEYBOARD#DESIGN_FEATURES
KEYBOARD#GENERAL
KEYBOARD#OPERATION_PERFORMANCE
LAPTOP#DESIGN_FEATURES
LAPTOP#GENERAL
LAPTOP#OPERATION_PERFORMANCE
LAPTOP#PRICE
LAPTOP#QUALITY
LAPTOP#USABILITY
LOCATION#GENERAL
LOCATION#OPERATION_PERFORMANCE
MULTIMEDIA_DEVICES#OPERATION_PERFORMANCE
OS#DESIGN_FEATURES
OS#GENERAL
OS#OPERATION_PERFORMANCE
OS#USABILITY
PAYMENT#GENERAL
POWER_SUPPLY#GENERAL
POWER_SUPPLY#OPERATION_PERFORMANCE
RESTAURANT#GENERAL
SERIES#GENERAL
SERVICE#GENERAL
SHIPPING#GENERAL
SHIPPING#OPERATION_PERFORMANCE
SINCE#GENERAL
SINCE#OPERATION_PERFORMANCE
SMS#GENERAL
SMS#OPERATION_PERFORMANCE
SOFTWARE#DESIGN_FEATURES
SOFTWARE#GENERAL


In [22]:
import pandas as pd
import ast
from collections import defaultdict

# Step 1: Parse stringified list columns
def safe_parse(col):
    try:
        return ast.literal_eval(str(col))
    except:
        return []

df['aspects'] = df['aspects'].apply(safe_parse)
df['opinions'] = df['opinions'].apply(safe_parse)
df['categories'] = df['categories'].apply(safe_parse)

# Step 2: Extract and group aspects/opinions by category
category_map = defaultdict(lambda: {"aspects": set(), "opinions": set()})

for _, row in df.iterrows():
    for aspect, opinion, category in zip(row['aspects'], row['opinions'], row['categories']):
        category_map[category]["aspects"].add(aspect)
        category_map[category]["opinions"].add(opinion)

# Step 3: Format as a list-based DataFrame
grouped_data = []
for category, items in category_map.items():
    grouped_data.append({
        "category": category,
        "aspects": sorted(list(items["aspects"])),
        "opinions": sorted(list(items["opinions"]))
    })

result_df = pd.DataFrame(grouped_data)

# Optional: View sample
result_df.head()


,category,aspects,opinions
0,LAPTOP#GENERAL,"[Amaing Features, Experience, HBL, NULL, Rubbi...","[5, Amaing, Amazing, Awesome, Bad, Best, DISap..."
1,SUPPORT#GENERAL,"[AIOU fee option, Al Habib bank network, Atm C...","[Awesome, BEST, Bad experience, Best, Excellen..."
2,SOFTWARE#GENERAL,"['Forget Password' option, APP, Aap, Account S...","[3rd class, Amazing, Awesome, Awsome, Bad, Bes..."
3,SERVICE#GENERAL,"[Branch, HBL, HBL je service, HBL service, HBL...","[3rd class, ANNOYING, Amazing, Bad, Bad bad, E..."
4,SOFTWARE#OPERATION_PERFORMANCE,"['_oneByteString', 3rd class app, 3rd class ap...","[3rd Class, Alot, Application Is Asking For Up..."


In [25]:
result_df.to_csv("cats.csv")

In [35]:
# Fix parsing of lists in 'aspects' and 'opinions' columns
def safe_parse_list(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception:
            return []
    elif isinstance(x, list):
        return x
    else:
        return []

# Define bank-app-specific category mapping (replacing generic or unrelated categories)
bank_category_map = {
    "LAPTOP#GENERAL": "APP#GENERAL_EXPERIENCE",
    "SUPPORT#GENERAL": "CUSTOMER_SUPPORT",
    "SOFTWARE#GENERAL": "APP#FEATURES",
    "SERVICE#GENERAL": "BRANCH_SERVICE",
    "SOFTWARE#OPERATION_PERFORMANCE": "APP#PERFORMANCE",
    "LAPTOP#OPERATION_PERFORMANCE": "DEVICE#PERFORMANCE",
    "LAPTOP#USABILITY": "APP#USABILITY",
    "OS#OPERATION_PERFORMANCE": "SYSTEM#PERFORMANCE",
    "HARDWARE#OPERATION_PERFORMANCE": "BIOMETRIC#PERFORMANCE",
    "HARDWARE#GENERAL": "BIOMETRIC#GENERAL",
    "POWER_SUPPLY#GENERAL": "UTILITIES#POWER_ISSUE",
    "RESTAURANT#GENERAL": "LOCATION_FEEDBACK_MISC",
    "KEYBOARD#OPERATION_PERFORMANCE": "BIOMETRIC#LOGIN_ISSUES",
    "LAPTOP#DESIGN_FEATURES": "UI_UX#DESIGN",
    "SINCE#GENERAL": "ATM_SERVICE",
    "SOFTWARE#USABILITY": "APP#USABILITY_FEATURES",
    "AMBIENCE#GENERAL": "ENVIRONMENT_FEEDBACK",
    "COSTS#GENERAL": "FEES_CHARGES",
    "COMPANY#GENERAL": "BANK_REPUTATION",
    "OS#GENERAL": "SYSTEM_UI",
    "SUPPORT#USABILITY": "CARD_SETTINGS_SUPPORT",
    "SERIES#GENERAL": "SERIES_FEEDBACK_MISC",
    "POWER_SUPPLY#OPERATION_PERFORMANCE": "UTILITIES#PERFORMANCE",
    "SHIPPING#GENERAL": "DELIVERY_RELATED_FEEDBACK",
    "DISPLAY#GENERAL": "SCREEN#GENERAL_FEEDBACK",
    "FOOD#QUALITY": "MISC#APP_FEEDBACK",
    "SINCE#OPERATION_PERFORMANCE": "TRANSACTION_SPEED",
    "SMS#OPERATION_PERFORMANCE": "EMAIL_SMS_SERVICE",
    "LAPTOP#PRICE": "APP#COST_FEEDBACK",
    "CURRENCY#OPERATION_PERFORMANCE": "BALANCE#FUNCTIONALITY",
    "SUPPORT#OPERATION_PERFORMANCE": "SUPPORT#PERFORMANCE",
    "Font#OPERATION_PERFORMANCE": "UI_FONT#ISSUE",
    "OS#DESIGN_FEATURES": "SYSTEM#DESIGN_FEATURES",
    "LOCATION#OPERATION_PERFORMANCE": "LOCATION_FEATURES",
    "KEYBOARD#GENERAL": "LOGIN_SECURITY_ISSUES",
    "LOCATION#GENERAL": "LIGHTING_OR_LOCATION_ISSUE",
    "DISPLAY#USABILITY": "APP#TABLET_VIEW_ISSUE",
    "SHIPPING#OPERATION_PERFORMANCE": "PAYMENT#STUCK",
    "SOFTWARE#DESIGN_FEATURES": "APP#DESIGN_FEEDBACK",
    "PAYMENT#GENERAL": "TAX_CHARGES_FEEDBACK",
    "LAPTOP#QUALITY": "APP#QUALITY_FEEDBACK",
    "KEYBOARD#DESIGN_FEATURES": "PASSWORD#FORMAT_ISSUES",
    "SUPPORT#QUALITY": "SUPPORT#RESPONSE",
    "COMplaint#OPERATION_PERFORMANCE": "COMPLAINT_PORTAL#FUNCTIONALITY",
    "MULTIMEDIA_DEVICES#OPERATION_PERFORMANCE": "CAMERA#PERFORMANCE",
    "CNIC#GENERAL": "CNIC#RELATED_FEEDBACK",
    "DRINKS#OPERATION_PERFORMANCE": "ACCOUNT_RECOVERY#ISSUES",
    "OS#USABILITY": "UI_COLOR#USABILITY",
    "DISPLAY#OPERATION_PERFORMANCE": "SCREEN#ISSUES",
    "SMS#GENERAL": "SMS#OTP_ISSUE",
    "CURRENCY#GENERAL": "DIGITAL_CURRENCY_FEEDBACK",
    "": "UNCLASSIFIED_ISSUES",
    "SOFTWARE#PRICE": "APP#PRICING_FEEDBACK",
    "CHARTER#GENERAL": "FEES_POLICY_COMPLAINTS",
}


def map_categories(raw_category_list):
    return [bank_category_map.get(cat.strip(), "Miscellaneous") for cat in raw_category_list]

def parse_and_map(df):
    df["categories"] = df["categories"].apply(safe_parse_list)
    df["mapped_categories"] = df["categories"].apply(map_categories)
    return df

df = parse_and_map(df)
df.head(50)
df.to_csv("

,Unnamed: 0,app,content,score,at,aspects,opinions,sentiments,categories,mapped_categories
0,0,Al Habib,good,5,2025-07-11 10:24:50,[NULL],[good],['positive'],[LAPTOP#GENERAL],[APP#GENERAL_EXPERIENCE]
1,1,Al Habib,Its useless i am trying to register my account...,1,2025-07-11 05:53:23,"[email error, helpline]","[useless, NULL]","['negative', 'negative']","[SUPPORT#GENERAL, SUPPORT#GENERAL]","[CUSTOMER_SUPPORT, CUSTOMER_SUPPORT]"
2,2,Al Habib,app is good but I have an issue now I'm living...,5,2025-07-10 23:28:19,[app],[good],['positive'],[SOFTWARE#GENERAL],[APP#FEATURES]
3,3,Al Habib,bank is good coie sarvis,5,2025-07-10 21:49:23,[bank],[good],['positive'],[SUPPORT#GENERAL],[CUSTOMER_SUPPORT]
4,4,Al Habib,There is so much wrong about this net banking ...,1,2025-07-10 18:01:09,"[net banking app, biometric problem, BAHL Net ...","[wrong, NULL, worst decision]","['negative', 'negative', 'negative']","[SOFTWARE#GENERAL, SOFTWARE#GENERAL, SOFTWARE#...","[APP#FEATURES, APP#FEATURES, APP#FEATURES]"
5,5,Al Habib,good app and fast service,5,2025-07-10 17:44:30,"[app, service]","[good, fast]","['positive', 'positive']","[SOFTWARE#GENERAL, SERVICE#GENERAL]","[APP#FEATURES, BRANCH_SERVICE]"
6,6,Al Habib,I can't tell what that was? they don't have an...,1,2025-07-10 12:29:28,[app],[NULL],['negative'],[SOFTWARE#OPERATION_PERFORMANCE],[APP#PERFORMANCE]
7,7,Al Habib,My app is not working since 4 days. i have a b...,1,2025-07-10 11:13:18,[app],[NULL],['negative'],[SOFTWARE#OPERATION_PERFORMANCE],[APP#PERFORMANCE]
8,8,Al Habib,Worst Experience app never work properly,1,2025-07-10 02:29:57,[Experience app],[Worst],['negative'],[SOFTWARE#OPERATION_PERFORMANCE],[APP#PERFORMANCE]
9,9,Al Habib,stupid not working since 2 days,2,2025-07-09 23:28:20,[NULL],[stupid],['negative'],[LAPTOP#OPERATION_PERFORMANCE],[DEVICE#PERFORMANCE]
